# 7. Weather Features


This notebook pulls historical weather data for each individual people monitoring sites making use of meteostat package.

https://meteostat.net/en/blog/obtain-weather-data-any-location-python



In [ ]:
%load_ext autoreload
%autoreload 2

from model_packages import *

# All the variables are defined in the Config file
from model_config import *

from model_utils import *

In [ ]:
os.chdir('..')

# Format People Counter site data

In [ ]:
# read counter locations

loctns=gpd.read_file(data_folder+'accessibility.shp')
loctns=loctns[loctns['geom_type']=='5km buffer'].reset_index(drop=True)

loctns.geometry=loctns.geometry.centroid

loctns=loctns.to_crs(crs_deg)

# reformat the counter dataset
indx=0
all_weather_site=[]
for sites in list(loctns['counter']):  
   lat=loctns[loctns['counter']==sites].geometry.y[indx]
   lon=loctns[loctns['counter']==sites].geometry.x[indx]
   pnt=Point(lat,lon)
   all_weather_site.append([sites,lat,lon])
   indx+=1

# Extract Weather data

In [ ]:
# Get weather data from the nearest monitoring station of each site
approxmt_wthr_df=[]
for nrst_stsns in range(len(all_weather_site)):
    # Get nearby weather stations
    stations = Stations()
    # select site from people counters
    stn_loc=all_weather_site[nrst_stsns][1:][0],all_weather_site[nrst_stsns][1:][1]
    
    #select nearby weather stations 
    stations = stations.nearby(stn_loc[0],stn_loc[1])
    station = stations.fetch(1)
    st_wthr_loc= Point(station['latitude'][0],station['longitude'][0])
    
    # extract weatger data between 2020-01-01 and 2022-12-31
    data = Monthly(st_wthr_loc, start, end)
    
    data = data.fetch()
    
    data['site']=all_weather_site[nrst_stsns][0]
    
    if len(data)==0:
        print(station)
    
    # append dara to dataframe
    approxmt_wthr_df.append(data)

In [ ]:
# Gather all weather data for each site
approxmt_wthr_df=pd.concat(approxmt_wthr_df)

weather_df=approxmt_wthr_df.reset_index()[['time','tavg','site']]

weather_df=weather_df.groupby(['site','time'])['tavg'].mean().reset_index()

# split into list of subset dataframes one for each site
weather_df_splt =[x for _, x in weather_df.groupby('site')]



#CJ: Is this the best way to deal with Nans ?
weather_df_splt=[x.dropna(how='all').fillna(method="ffill").fillna(method='bfill') for x in weather_df_splt]

# concatenate subsets after dealing with NaNs
weather_df=pd.concat(weather_df_splt)

# data wrangling 
weather_df.rename(columns={'time':'Date'},inplace=True)
weather_df['Date']=weather_df['Date'].dt.to_period('M')
# save and export weather data
weather_df.to_pickle(data_folder+'weather_df.pkl')



In [ ]:
# check for null values
print('Any null values: {}'.format(weather_df.isna().sum(axis=0)))

# Visualisation

In [ ]:
rndm_sites=random.sample(list(weather_df['site'].unique()), 5)

In [ ]:
for site in rndm_sites:
    weather_df[weather_df['site']==site].set_index('Date')['tavg'].plot(label=site)
    plt.legend()
    plt.ylabel('tavg')

